# Map of all US Households with Children and Elderly and Distance to Tanks Range Slider

This visualization uses a the Cuxfilter library and GPUs to plot households and tanks on a dashboard, with range slider feature to show households that are a certain range of distance away from the nearest tank. There are also multiselect charts to view households within a certain age group and households within a specific distance category. In this visualization, we are only looking at households with children and elderly.

### Import statements

In [1]:
import geopandas as gpd
import numpy as np

import cuxfilter

import cudf
from bokeh.palettes import Spectral11

import holoviews as hv
import pandas as pd

### Reading in household and proximity data (pre-processed)
This is a preprocessed file with distance between households across the whole US (that have both children and eldery people) and tanks already calculated in miles. This dataframe also includes information on age group of the head of household, the latitude and longitudes of the tanks and households, distance category, and tank_type.

The ```lat_3857``` and ```lon_3857``` coordinates will be the points we plot on our cuxfilter dashboard, and the rest of the variables are used for the range slider and multiselect tools the user can interact with.

In [2]:
df = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/all_dist_df_final.parquet')
df

,age_code,lat_3857,lon_3857,distance_mi,distance_category,age_group,tank_type
0,K,-8.495076e+06,5.104329e+06,29.813838,4,2,None
1,J,-8.495201e+06,5.099593e+06,28.258237,4,1,None
2,M,-8.503119e+06,5.095542e+06,29.790321,4,4,None
3,M,-8.503638e+06,5.104549e+06,32.673770,4,4,None
4,L,-8.503195e+06,5.095720e+06,29.868401,4,3,None
...,...,...,...,...,...,...,...
12008076,None,-1.167972e+07,4.833652e+06,215.000000,0,0,narrow_closed_roof_tank
12008077,None,-1.167971e+07,4.833633e+06,215.000000,0,0,narrow_closed_roof_tank
12008078,None,-1.167971e+07,4.833691e+06,215.000000,0,0,narrow_closed_roof_tank
12008079,None,-1.167971e+07,4.833535e+06,215.000000,0,0,narrow_closed_roof_tank


### Transforming to cuxfilter dataframe

This transforms the pandas dataframe into a cuDF dataframe, then from a cuDF dataframe into a Cuxfilter dataframe. This makes it possible to plot these dataframes using the Cuxfilter library.

In [3]:
cdf = cudf.DataFrame.from_pandas(df)

In [4]:
cux_df = cuxfilter.DataFrame.from_dataframe(cdf)

### Defining label maps
The Datashader plotting library that Cuxfilter uses to create our visualization through the use of Graphical Processing Units (GPUs) is optimized for working with large dataframes. Here, we're plotting over 12 million of them. However, an aspect of Datashader is that it only takes numerical inputs when creating the range slider and multiselect charts. This means that instead of being able to categorize each household by age group by labelling it with ```strings``` like ```'70-74```, we must label it numerically. Hence, our column ```age_group``` has numerical indicators. ```0``` indicates that the point is a tank, and the rest indicate that the point is a household and the head of household is of a specific age group seen in the ```label_age_group``` dictionary below. The same structure is true for our ```distance_category``` column. For the ```distance_category``` column, ```0``` indicates that the point is a tank, and the rest indicate that the point is a household which is a specific distance category away from a tank as seen in the ```label_distance_category``` dictionary. 

The label maps below associate each numerical value in our dataframe to a ```string``` label which is displayed on the range slider and multiselect. The ```colors``` list provides the hex codes for the coloring of each point in the map when it is displayed. 

In [5]:
label_age_group = {0: 'Tank', 1: '65+ (inferred)', 
             2: '65-69 (reported)', 3: '70-74', 
             4: '75+ (reported)'}

In [6]:
label_distance_category = {0: 'Tank', 1: '0.5 mi away from tank', 
             2: '1 mi away from tank', 3: '5 mi away from tank', 
             4: 'More than 5 mi away from tank'}

In [7]:
colors = list(Spectral11)
colors.reverse()

### Defining Cuxfilter Charts

This code defines the charts for our interactive dashboard. The ```points``` chart is the main map with households and tanks plotted. We are using latitude and longitude coordinates in the EPSG 3857 coordinate system, as it is the one used by the Cuxfilter library. The points for tanks and households are colored differently by setting the ```aggregate_col``` parameter as the ```distance_mi``` column in our dataframe. The ```aggregate_fn``` parameter, set to either ```max```, ```min``` or ```mean``` specifies which aggregation of the data to perform when coloring the points. 

This column has four categories, as described above, and the ```colors``` list has 11 different colors, used to color each point by its distance to the nearest tank as seen in the ```distance_mi``` column. 

The next three charts define the interactive range sliders and multiselects the user can interact with. In each of these lines, the ```.multi_select``` specifies that the chart is a multiselect chart, while the ```.range_slider``` specifies that the chart is a range slider. In each of these chart definitions, we specify the column name from our dataframe that the chart should pull from, and the label map Cuxfilter should use to create the chart. For example, the ```distance_category``` chart is a multiselect chart that pulls from the ```distance_category``` column. The options on that multiselect chart are labelled according to the ```label_map_distance``` label map.

In [8]:
points = cuxfilter.charts.scatter(x='lat_3857', y='lon_3857', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'mean', aggregate_col = 'distance_mi', tile_provider="CartoDark", title = 'Households with Elderly and Children in Proximity to Tanks',
                                  legend = True)

age_group = cuxfilter.charts.multi_select('age_group', label_map=label_age_group)

distance_category = cuxfilter.charts.multi_select('distance_category', label_map=label_distance_category)

distance = cuxfilter.charts.range_slider('distance_mi')

Finally, we use the ```.dashboard``` method to put these charts together as an interactive dashboard for the user. We first specify the main charts which will be displayed in the ```layout``` we choose, then specify the charts that will be displayed on the dashboard's ```sidebar```.

In [9]:
d = cux_df.dashboard([points], sidebar = [age_group, distance_category, distance])

### Displaying interactive dashboard

Running the commands below displays the interactive dashboard. The user can use the multiselect charts to view specific subsets of points, and use ```ctrl``` click to view multiple catgories in one multiselect at a time. The user can also use the range slider chart to view households within a certain distance range from a storage tank.

Using these interactive tools creates a copy of the data every time a user interacts with them, which may cause a memory allocation error. You can run ```nvidia-smi``` to see how much memory you are using on the GPUs.

In [10]:
d.show()
d.app(sidebar_width=290) # run the dashboard within the notebook cell

Dashboard running at port 49405


Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=290)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=12008081)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='age_group_multi_select', width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, '65+ (inferred...}, sizing_mode='stretch_both', value=[''], width=400)
        [2] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='distance_category_multi_s..., width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, '0.5 mi away f...}, sizing_mode='stretch_both', value=[''], width=400)
        [3] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='distance_mi_range_slider'..., width=400)
            [0] RangeSlider(end=215.0, sizing_mode='stretch_both', start=0.000592927239086057, value=(0.000592927239086057, ..., value_end=215.0, value_start=0.000592927239086057)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=6, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Households with E..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')